In [35]:
import pandas as pd
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer 

In [15]:
client = MongoClient()

In [16]:
client.database_names()

['local', 'yelp']

In [17]:
db = client.yelp

In [18]:
collection = db.reviews

In [43]:
funnies = collection.find({'votes.funny':{'$gt':10}})

In [44]:
funnies.count()

7303

In [65]:
reviews[1]

'MEAN CUSTOMERS -ALOOF WORKERS+GOOD FOOD = CRAZY!\n\nOfficial Rant of an Angry Woman:\n\nFirst off - I shop at "Whole Paycheck" cause my household eats organic and whether I like it or not WF has the best organic food prices in my neck of the woods. IT\'S STILL GOD DAMN RIDICULOUS EXPENSIVE but I take my family units nutrition and health seriously! WORD!!\n\nDownhill from here...\n\nTHE CUSTOMERS AND PARKING LOT ARE MORE BAT SHIT CRAZY THAN MENTAL HOSPITALS I\'VE WORKED AT!\n\nI have watched more rich bitch elderly women NEARLY murder pedestrians and crash into cars trying to park their Lincoln Continentals in that parking lot than I\'d ever care to witness!\n\nAlso - I am a mother with a young child and have been SHOVED by above mentioned rich bitches while reaching for products out of the freezer cases WITH MY CHILD IN MY ARMS! I briefly entertained SHANKING said bitch but alas prison orange is not in my color wheel!\n\nThe customers are the worst human beings on earth (::thinks abou

In [51]:
reviews = []

In [52]:
for i in funnies:
    reviews.append(i['text'])

In [67]:
vectorizer = CountVectorizer(stop_words='english',min_df=2)

In [68]:
vectorizer.fit(reviews)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [69]:
vectorizer.get_feature_names()

['00',
 '000',
 '007',
 '00am',
 '00pm',
 '01',
 '02',
 '03',
 '04',
 '05',
 '05am',
 '07',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '100k',
 '100s',
 '100th',
 '100x',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '107th',
 '108',
 '109',
 '10am',
 '10k',
 '10mins',
 '10p',
 '10pm',
 '10th',
 '10x',
 '11',
 '110',
 '111',
 '112',
 '115',
 '118',
 '119',
 '11a',
 '11am',
 '11pm',
 '11pts',
 '11th',
 '12',
 '120',
 '1200',
 '120s',
 '122',
 '123',
 '125',
 '126',
 '128',
 '12am',
 '12i0buxgq',
 '12oz',
 '12pm',
 '12th',
 '13',
 '130',
 '134',
 '135',
 '13th',
 '14',
 '140',
 '142',
 '145',
 '149',
 '14oz',
 '14th',
 '15',
 '150',
 '1500',
 '151',
 '153',
 '155',
 '158',
 '159',
 '15am',
 '15min',
 '15mins',
 '15minutes',
 '15pm',
 '15th',
 '16',
 '160',
 '164',
 '165',
 '167',
 '168',
 '169',
 '16oz',
 '16th',
 '17',
 '170',
 '175',
 '17th',
 '18',
 '180',
 '1800',
 '182',
 '185',
 '18oz',
 '18th',
 '19',
 '190',
 '1920',
 '1920s',
 '1930',
 '1940',
 '1940s',
 '1941',
 '1945',
